# 清理超大 Highres 图片及对应 Lowres

为避免推理时显存溢出，应该从 **highres** 端控制：删除 **highres 总像素超过 25.6M**（即 1600×1000×4×4，对应 4× 放大后的 LR 为 1.6M 像素）的图片，并同时删除其对应的 lowres 图。

配对规则：与 `degradation_pipeline.ipynb` / `train_esrgan_independent.ipynb` 一致，lowres 与 highres 通过 **相同文件名** 一一对应（如 `372159.jpg` 在 `dataset/lowres/original/` 与 `dataset/highres/original/` 中各一份）。

> 注意：这个 notebook 的旧版本已经按 lowres>1.6M 删除过一遍，这里是 **新的规则：直接按 highres>25.6M 检查并删除**。

In [5]:
import os
import glob

# 与 degradation_pipeline / 训练脚本保持一致
LR_DIR = 'dataset/lowres/original'
HR_DIR = 'dataset/highres/original'

# Highres 总像素上限：超过则删除（1600*1000*4*4 = 25.6M）
MAX_PIXELS = 25_600_000

In [6]:
from PIL import Image

def get_image_pixels(path):
    """获取图片总像素（宽*高），失败返回 None。"""
    try:
        with Image.open(path) as im:
            w, h = im.size
            return w * h
    except Exception as e:
        print(f"  无法读取: {path} -> {e}")
        return None

# 获取所有 highres 图片路径（与 degradation 一致：jpg/jpeg/png）
hr_paths = sorted(glob.glob(os.path.join(HR_DIR, '*.[jp][pn]*g')))
print(f"Highres 目录中共 {len(hr_paths)} 张图片。")

to_remove = []  # [(lr_path, hr_path, pixels), ...]
for hr_path in hr_paths:
    pixels = get_image_pixels(hr_path)
    if pixels is None:
        continue
    if pixels > MAX_PIXELS:
        filename = os.path.basename(hr_path)
        lr_path = os.path.join(LR_DIR, filename)
        to_remove.append((lr_path, hr_path, pixels))

print(f"其中 highres 总像素 > {MAX_PIXELS:,} 的共有 {len(to_remove)} 张，将连同对应 lowres 一并删除。")

Highres 目录中共 444 张图片。
其中 highres 总像素 > 25,600,000 的共有 10 张，将连同对应 lowres 一并删除。


In [7]:
if not to_remove:
    print("没有需要删除的图片。")
else:
    print("待删除列表（highres 路径 | 总像素）：")
    for lr_path, hr_path, pixels in to_remove:
        name = os.path.basename(hr_path)
        with Image.open(hr_path) as im:
            w, h = im.size
        print(f"  {name}: {w}*{h} = {pixels:,} 像素")

待删除列表（highres 路径 | 总像素）：
  374729.png: 12074*4334 = 52,328,716 像素
  378937.png: 8192*5074 = 41,566,208 像素
  379479.png: 7000*5300 = 37,100,000 像素
  381639.png: 7163*4632 = 33,179,016 像素
  382338.png: 7000*4591 = 32,137,000 像素
  384731.png: 6314*4465 = 28,192,010 像素
  386490.png: 7680*4320 = 33,177,600 像素
  395724.png: 7680*4320 = 33,177,600 像素
  396358.png: 8667*5006 = 43,387,002 像素
  399218.png: 7439*4087 = 30,403,193 像素


In [8]:
deleted_lr = 0
deleted_hr = 0

for lr_path, hr_path, pixels in to_remove:
    if os.path.isfile(lr_path):
        os.remove(lr_path)
        deleted_lr += 1
        print(f"已删除 lowres:  {os.path.basename(lr_path)}")
    if os.path.isfile(hr_path):
        os.remove(hr_path)
        deleted_hr += 1
        print(f"已删除 highres: {os.path.basename(hr_path)}")
    else:
        print(f"  警告: 未找到对应 highres: {os.path.basename(hr_path)}")

print(f"\n完成。共删除 lowres {deleted_lr} 张、highres {deleted_hr} 张。")

已删除 highres: 374729.png
已删除 highres: 378937.png
已删除 highres: 379479.png
已删除 highres: 381639.png
已删除 highres: 382338.png
已删除 highres: 384731.png
已删除 highres: 386490.png
已删除 highres: 395724.png
已删除 highres: 396358.png
已删除 highres: 399218.png

完成。共删除 lowres 0 张、highres 10 张。
